In [4]:
pip install openmeteo-requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.


In [26]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [10]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [28]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 60.1695,
	"longitude": 24.9354,
	"start_date": "2010-01-01",
	"end_date": "2019-12-31",
	"hourly": ["temperature_2m", "rain", "snowfall", "sunshine_duration", "soil_temperature_0_to_7cm", "weather_code", "relative_humidity_2m", "precipitation"],
	"timezone": "Europe/Berlin",
	"temporal_resolution": "hourly_6"
}
responses = openmeteo.weather_api(url, params=params)

In [30]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 60.140594482421875°N 24.965036392211914°E
Elevation 12.0 m asl
Timezone b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0 7200 s


In [32]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(2).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(3).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(4).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(5).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(6).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(7).ValuesAsNumpy()

In [34]:
hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

In [36]:
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["weather_code"] = hourly_weather_code
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)

In [46]:
hourly_dataframe.to_csv('./export/weather2010_2019_helsinki.csv', sep=',', encoding='utf-8', index=False, header=True)

In [66]:
hourly_dataframe.head()

,date,temperature_2m,rain,snowfall,sunshine_duration,soil_temperature_0_to_7cm,weather_code,relative_humidity_2m,precipitation
0,2009-12-31 22:00:00+00:00,-10.6805,0.0,0.07,0.0,-1.8805,71.0,87.295601,0.0
1,2010-01-01 04:00:00+00:00,-9.9805,0.0,0.21,0.0,-1.8805,71.0,86.323875,0.1
2,2010-01-01 10:00:00+00:00,-10.9305,0.0,0.00,0.0,-1.8805,3.0,84.836220,0.0
3,2010-01-01 16:00:00+00:00,-13.2805,0.0,0.00,0.0,-1.8805,3.0,84.548553,0.0
4,2010-01-01 22:00:00+00:00,-16.0805,0.0,0.00,0.0,-1.8805,3.0,84.911079,0.0


In [50]:
print(hourly_dataframe.shape[0])

14608


In [64]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14608 entries, 0 to 14607
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   date                       14608 non-null  datetime64[ns, UTC]
 1   temperature_2m             14608 non-null  float32            
 2   rain                       14608 non-null  float32            
 3   snowfall                   14608 non-null  float32            
 4   sunshine_duration          14608 non-null  float32            
 5   soil_temperature_0_to_7cm  14608 non-null  float32            
 6   weather_code               14608 non-null  float32            
 7   relative_humidity_2m       14608 non-null  float32            
 8   precipitation              14608 non-null  float32            
dtypes: datetime64[ns, UTC](1), float32(8)
memory usage: 570.8 KB
